## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santiago Del Estero,AR,-27.7951,-64.2615,58.78,29,1,10.11,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,35.26,80,40,2.30,scattered clouds
2,2,Rikitea,PF,-23.1203,-134.9692,70.48,61,47,18.75,scattered clouds
3,3,Lagoa,PT,39.0500,-27.9833,74.44,85,99,11.65,light rain
4,4,Tasiilaq,GL,65.6145,-37.6368,42.96,74,99,1.23,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]
preferred_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
14,14,La Orilla,MX,17.9833,-102.2333,82.24,73,15,3.98,few clouds
19,19,Poso,ID,-1.3959,120.7524,83.34,59,86,1.74,overcast clouds
22,22,Banjar,ID,-8.1900,114.9675,82.31,57,14,3.09,few clouds
27,27,Hilo,US,19.7297,-155.0900,88.29,85,100,11.50,overcast clouds
30,30,Kavaratti,IN,10.5669,72.6420,80.17,84,100,23.58,light rain
33,33,Shingu,JP,33.7333,135.9833,82.53,79,77,7.23,moderate rain
34,34,Alofi,NU,-19.0595,-169.9187,80.49,78,100,12.66,overcast clouds
40,40,Tabuk,PH,17.4189,121.4443,81.28,81,99,4.07,overcast clouds
41,41,Hithadhoo,MV,-0.6000,73.0833,81.99,74,100,12.44,overcast clouds
43,43,Palmer,US,42.1584,-72.3287,82.44,74,40,5.75,scattered clouds


In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities.count()

City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,La Orilla,MX,82.24,few clouds,17.9833,-102.2333,
19,Poso,ID,83.34,overcast clouds,-1.3959,120.7524,
22,Banjar,ID,82.31,few clouds,-8.1900,114.9675,
27,Hilo,US,88.29,overcast clouds,19.7297,-155.0900,
30,Kavaratti,IN,80.17,light rain,10.5669,72.6420,
33,Shingu,JP,82.53,moderate rain,33.7333,135.9833,
34,Alofi,NU,80.49,overcast clouds,-19.0595,-169.9187,
40,Tabuk,PH,81.28,overcast clouds,17.4189,121.4443,
41,Hithadhoo,MV,81.99,overcast clouds,-0.6000,73.0833,
43,Palmer,US,82.44,scattered clouds,42.1584,-72.3287,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")            

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,La Orilla,MX,82.24,few clouds,17.9833,-102.2333,Baymont by Wyndham Lazaro Cardenas
19,Poso,ID,83.34,overcast clouds,-1.3959,120.7524,Hotel 77 Poso
22,Banjar,ID,82.31,few clouds,-8.1900,114.9675,The Hamsa Resort
27,Hilo,US,88.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
30,Kavaratti,IN,80.17,light rain,10.5669,72.6420,Green leaf
33,Shingu,JP,82.53,moderate rain,33.7333,135.9833,Shingu UI Hotel
34,Alofi,NU,80.49,overcast clouds,-19.0595,-169.9187,Taloa Heights
40,Tabuk,PH,81.28,overcast clouds,17.4189,121.4443,Golden Berries Hotel and Convention
41,Hithadhoo,MV,81.99,overcast clouds,-0.6000,73.0833,Scoop Guest House
43,Palmer,US,82.44,scattered clouds,42.1584,-72.3287,Wedgewood Motel


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))